In [2]:
!pip install langchain-community ollama selenium beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3

In [4]:
!pip install colorama

In [5]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re
import json
from typing import List, Dict, Optional
from dataclasses import dataclass, field
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from urllib.parse import urljoin, urlparse
from collections import deque
from langchain_community.llms import Ollama
import colorama
from colorama import Fore, Back, Style

# Initialize colorama for colored output
colorama.init()

@dataclass
class RealEstateAgent:
    name: str = ""
    title: str = ""
    company: str = ""
    email: str = ""
    phone: str = ""
    location: str = ""
    social_media: Dict[str, str] = field(default_factory=dict)
    website: str = ""
    bio: str = ""
    specialties: List[str] = field(default_factory=list)
    languages: List[str] = field(default_factory=list)

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items() if v}

    def __str__(self):
        info = []
        if self.name:
            info.append(f"Name: {self.name}")
        if self.title:
            info.append(f"Title: {self.title}")
        if self.company:
            info.append(f"Company: {self.company}")
        if self.email:
            info.append(f"Email: {self.email}")
        if self.phone:
            info.append(f"Phone: {self.phone}")
        if self.location:
            info.append(f"Location: {self.location}")
        if self.social_media:
            info.append("Social Media:")
            for platform, url in self.social_media.items():
                info.append(f"  - {platform}: {url}")
        if self.specialties:
            info.append(f"Specialties: {', '.join(self.specialties)}")
        if self.languages:
            info.append(f"Languages: {', '.join(self.languages)}")
        if self.bio:
            info.append(f"Bio: {self.bio[:200]}...")
        return "\n".join(info)

class VerboseRealEstateLeadScraper:
    def __init__(self, max_depth: int = 1, max_results: int = 5, model_name: str = "llama2"):
        self.max_depth = max_depth
        self.max_results = max_results
        self.llm = Ollama(model=model_name)
        self.visited_urls = set()
        self.visited_domains = set()
        self.setup_logging()
        self.setup_patterns()
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def setup_patterns(self):
        self.patterns = {
            'email': re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'),
            'phone': re.compile(r'(?:\+?1[-.]?)?\s*(?:\([0-9]{3}\)|[0-9]{3})[-.]?[0-9]{3}[-.]?[0-9]{4}'),
            'social': {
                'linkedin': re.compile(r'linkedin\.com/(?:in|company)/[a-zA-Z0-9_-]+'),
                'facebook': re.compile(r'facebook\.com/[a-zA-Z0-9.]+'),
                'instagram': re.compile(r'instagram\.com/[a-zA-Z0-9_]+'),
                'twitter': re.compile(r'twitter\.com/[a-zA-Z0-9_]+')
            }
        }

    def print_debug(self, message: str, color: str = Fore.WHITE, prefix: str = ""):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"{color}[{timestamp}] {prefix}{message}{Style.RESET_ALL}")

    def get_base_domain(self, url: str) -> str:
        """Extract the base domain from a URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def google_search(self, query: str) -> List[Dict]:
        self.print_debug(f"Performing Google search for: {query}", Fore.CYAN, "🔍 ")
        query_encoded = urllib.parse.quote_plus(query)
        url = f"https://www.google.com/search?q={query_encoded}"

        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            search_results = soup.find_all('div', class_='g')

            results = []
            for result in search_results:
                title_element = result.find('h3')
                link_element = result.find('a')

                if title_element and link_element:
                    results.append({
                        'title': title_element.get_text(),
                        'link': link_element['href']
                    })
                    self.print_debug(f"Found result: {title_element.get_text()}", Fore.GREEN, "  ✓ ")

            self.print_debug(f"Found {len(results)} search results", Fore.CYAN, "📊 ")
            return results
        except Exception as e:
            self.print_debug(f"Error in Google search: {str(e)}", Fore.RED, "❌ ")
            return []

    def get_internal_links(self, url: str, html_content: str) -> List[str]:
        self.print_debug(f"Scanning for internal links on: {url}", Fore.YELLOW, "🔍 ")
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            base_domain = self.get_base_domain(url)
            internal_links = set()

            relevant_patterns = [
                '/agent/', '/about/', '/team/', '/contact/',
                '/profile/', '/bio/', '/realtor/', '/broker/'
            ]

            for link in soup.find_all('a', href=True):
                href = link['href']
                absolute_url = urljoin(url, href)

                if self.get_base_domain(absolute_url) == base_domain:
                    if any(pattern in absolute_url.lower() for pattern in relevant_patterns):
                        internal_links.add(absolute_url)
                        self.print_debug(f"Found relevant internal link: {absolute_url}", Fore.GREEN, "  ✓ ")

            self.print_debug(f"Found {len(internal_links)} relevant internal links", Fore.YELLOW, "📊 ")
            return list(internal_links)
        except Exception as e:
            self.print_debug(f"Error getting internal links: {str(e)}", Fore.RED, "❌ ")
            return []

    def extract_agent_info(self, url: str, html_content: str) -> Optional[RealEstateAgent]:
        self.print_debug(f"Extracting agent information from: {url}", Fore.MAGENTA, "🔍 ")
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            text_content = ' '.join(soup.stripped_strings)

            agent = RealEstateAgent()
            agent.website = url

            # Extract emails
            emails = self.patterns['email'].findall(text_content)
            if emails:
                agent.email = emails[0]
                self.print_debug(f"Found email: {agent.email}", Fore.GREEN, "  ✓ ")

            # Extract phone numbers
            phones = self.patterns['phone'].findall(text_content)
            if phones:
                phone = re.sub(r'[^\d]', '', phones[0])
                if len(phone) == 10:
                    agent.phone = f"({phone[:3]}) {phone[3:6]}-{phone[6:]}"
                    self.print_debug(f"Found phone: {agent.phone}", Fore.GREEN, "  ✓ ")

            # Extract social media
            for platform, pattern in self.patterns['social'].items():
                matches = pattern.findall(html_content)
                if matches:
                    agent.social_media[platform] = f"https://www.{matches[0]}"
                    self.print_debug(f"Found {platform} profile", Fore.GREEN, "  ✓ ")

            # Use LLM for detailed information
            self.print_debug("Using LLM to extract detailed information...", Fore.BLUE, "🤖 ")
            prompt = f"""
            Extract real estate agent information from this text:
            {text_content[:2000]}

            Return a JSON object with these fields:
            - name (full name of the agent)
            - title (professional title/role)
            - company (brokerage or company name)
            - location (city and state)
            - bio (brief professional description)
            - specialties (list of specializations)
            - languages (list of languages spoken)

            Only include information that is clearly present in the text.
            """

            try:
                llm_response = self.llm.invoke(prompt)
                agent_info = json.loads(llm_response)

                agent.name = agent_info.get('name', '')
                agent.title = agent_info.get('title', '')
                agent.company = agent_info.get('company', '')
                agent.location = agent_info.get('location', '')
                agent.bio = agent_info.get('bio', '')
                agent.specialties = agent_info.get('specialties', [])
                agent.languages = agent_info.get('languages', [])

                self.print_debug("Successfully extracted agent information", Fore.GREEN, "✓ ")

            except Exception as e:
                self.print_debug(f"Error parsing LLM response: {str(e)}", Fore.RED, "❌ ")

            if self.validate_agent(agent):
                self.print_debug("Agent information validated successfully.", Fore.GREEN, "✅ ")
                return agent
            else:
                self.print_debug("Agent information validation failed.", Fore.RED, "❌ ")
                return None
        except Exception as e:
            self.print_debug(f"Error extracting agent info: {str(e)}", Fore.RED, "❌ ")
            return None

    def validate_agent(self, agent: RealEstateAgent) -> bool:
        # Check if mandatory fields are filled
        return bool(agent.name and (agent.email or agent.phone))

    def scrape_leads(self, query: str) -> List[Dict]:
        leads = []
        search_results = self.google_search(query)[:self.max_results]

        with ThreadPoolExecutor(max_workers=5) as executor:
            futures = []
            for result in search_results:
                url = result['link']
                domain = self.get_base_domain(url)
                if domain not in self.visited_domains:
                    self.visited_domains.add(domain)
                    self.visited_urls.add(url)
                    futures.append(executor.submit(self.process_url, url, depth=1))

            for future in as_completed(futures):
                agent = future.result()
                if agent:
                    leads.append(agent.to_dict())

        return leads

    def process_url(self, url: str, depth: int) -> Optional[RealEstateAgent]:
        if depth > self.max_depth:
            return None

        self.print_debug(f"Processing URL: {url} (Depth: {depth})", Fore.YELLOW, "🔍 ")
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()

            agent_info = self.extract_agent_info(url, response.text)

            if not agent_info and depth < self.max_depth:
                internal_links = self.get_internal_links(url, response.text)
                for link in internal_links:
                    if link not in self.visited_urls:
                        link_domain = self.get_base_domain(link)
                        if link_domain not in self.visited_domains:
                            self.visited_urls.add(link)
                            internal_agent_info = self.process_url(link, depth + 1)
                            if internal_agent_info:
                                return internal_agent_info

            return agent_info
        except Exception as e:
            self.print_debug(f"Error processing URL {url}: {str(e)}", Fore.RED, "❌ ")
            return None

    def print_results(self, leads: List[Dict]):
        print("\n" + "="*50)
        print(f"{Fore.CYAN}LEAD GENERATION RESULTS{Style.RESET_ALL}")
        print("="*50)

        for i, lead in enumerate(leads, 1):
            print(f"\n{Fore.GREEN}Lead #{i}{Style.RESET_ALL}")
            print("-"*30)
            for key, value in lead.items():
                print(f"{key.capitalize()}: {value}")

        # Optionally, save the results to a JSON file
        self.save_results_to_json(leads)

    def save_results_to_json(self, leads: List[Dict], filename: str = 'real_estate_leads.json'):
        try:
            with open(filename, 'w') as f:
                json.dump(leads, f, indent=4)
            self.print_debug(f"Results saved to {filename}", Fore.BLUE, "💾 ")
        except Exception as e:
            self.print_debug(f"Error saving results: {str(e)}", Fore.RED, "❌ ")



In [7]:
search_query = "real estate agents in San Francisco"
scraper = VerboseRealEstateLeadScraper(max_depth=1, max_results=20)
leads = scraper.scrape_leads(search_query)
scraper.print_results(leads)

[2024-10-05 00:27:35] 🔍 Performing Google search for: real estate agents in San Francisco
[2024-10-05 00:27:36]   ✓ Found result: Find Realtors & Real Estate Agents in San Francisco, CA
[2024-10-05 00:27:36]   ✓ Found result: Real estate agents in San Francisco, CA - Zillow
[2024-10-05 00:27:36]   ✓ Found result: San Francisco Agents, 873 Compass Real Estate Agents
[2024-10-05 00:27:36]   ✓ Found result: Kinoko Real Estate: Top Producing San Francisco Realtors
[2024-10-05 00:27:36]   ✓ Found result: 884 Real Estate Agents in San Francisco, CA
[2024-10-05 00:27:36]   ✓ Found result: THE BEST 10 Real Estate Agents in SAN FRANCISCO, CA
[2024-10-05 00:27:36]   ✓ Found result: SF Real Estate Agent | Realtor San Francisco
[2024-10-05 00:27:36]   ✓ Found result: Realtors, San Francisco | Real Estate Agents
[2024-10-05 00:27:36]   ✓ Found result: Search for a Real Estate Agent in San Francisco, CA
[2024-10-05 00:27:36] 📊 Found 9 search results
[2024-10-05 00:27:36] 🔍 Processing URL: https://ww

In [9]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import re
import json
from typing import List, Dict, Optional
from dataclasses import dataclass, field
import logging
from collections import deque
import time
from urllib.parse import urljoin, urlparse
from langchain_community.llms import Ollama
import colorama
from colorama import Fore, Style

# Initialize colorama for colored output
colorama.init()

@dataclass
class RealEstateAgent:
    name: str = ""
    title: str = ""
    company: str = ""
    email: str = ""
    phone: str = ""
    location: str = ""
    social_media: Dict[str, str] = field(default_factory=dict)
    website: str = ""
    bio: str = ""
    specialties: List[str] = field(default_factory=list)
    languages: List[str] = field(default_factory=list)

    def to_dict(self):
        return {k: v for k, v in self.__dict__.items() if v}

    def __str__(self):
        info = []
        if self.name:
            info.append(f"Name: {self.name}")
        if self.title:
            info.append(f"Title: {self.title}")
        if self.company:
            info.append(f"Company: {self.company}")
        if self.email:
            info.append(f"Email: {self.email}")
        if self.phone:
            info.append(f"Phone: {self.phone}")
        if self.location:
            info.append(f"Location: {self.location}")
        if self.social_media:
            info.append("Social Media:")
            for platform, url in self.social_media.items():
                info.append(f"  - {platform}: {url}")
        if self.specialties:
            info.append(f"Specialties: {', '.join(self.specialties)}")
        if self.languages:
            info.append(f"Languages: {', '.join(self.languages)}")
        if self.bio:
            info.append(f"Bio: {self.bio[:200]}...")
        return "\n".join(info)

class VerboseRealEstateLeadScraper:
    def __init__(self, max_depth: int = 1, max_results: int = 5, model_name: str = "llama2"):
        self.max_depth = max_depth
        self.max_results = max_results
        self.llm = Ollama(model=model_name)
        self.visited_urls = set()
        self.setup_logging()
        self.setup_patterns()
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )
        self.logger = logging.getLogger(__name__)

    def setup_patterns(self):
        self.patterns = {
            'email': re.compile(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'),
            'phone': re.compile(r'(?:\+?1[-.]?)?\s*(?:\([0-9]{3}\)|[0-9]{3})[-.]?[0-9]{3}[-.]?[0-9]{4}'),
            'social': {
                'linkedin': re.compile(r'linkedin\.com/(?:in|company)/[a-zA-Z0-9_-]+'),
                'facebook': re.compile(r'facebook\.com/[a-zA-Z0-9.]+'),
                'instagram': re.compile(r'instagram\.com/[a-zA-Z0-9_]+'),
                'twitter': re.compile(r'twitter\.com/[a-zA-Z0-9_]+')
            }
        }

    def print_debug(self, message: str, color: str = Fore.WHITE, prefix: str = ""):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        print(f"{color}[{timestamp}] {prefix}{message}{Style.RESET_ALL}")

    def google_search(self, query: str) -> List[str]:
        self.print_debug(f"Performing Google search for: {query}", Fore.CYAN, "🔍 ")
        query_encoded = urllib.parse.quote_plus(query)
        url = f"https://www.google.com/search?q={query_encoded}"

        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            search_results = soup.find_all('div', class_='g')

            results = []
            for result in search_results:
                link_element = result.find('a')

                if link_element:
                    link = link_element['href']
                    results.append(link)
                    self.print_debug(f"Found result: {link}", Fore.GREEN, "  ✓ ")

            self.print_debug(f"Found {len(results)} search results", Fore.CYAN, "📊 ")
            return results[:self.max_results]
        except Exception as e:
            self.print_debug(f"Error in Google search: {str(e)}", Fore.RED, "❌ ")
            return []

    def get_base_domain(self, url: str) -> str:
        """Extract the base domain from a URL."""
        parsed_url = urlparse(url)
        return parsed_url.netloc

    def get_internal_links(self, url: str, html_content: str) -> List[str]:
        self.print_debug(f"Scanning for internal links on: {url}", Fore.YELLOW, "🔍 ")
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            base_domain = self.get_base_domain(url)
            internal_links = set()

            relevant_patterns = [
                '/agent/', '/about/', '/team/', '/contact/',
                '/profile/', '/bio/', '/realtor/', '/broker/'
            ]

            for link in soup.find_all('a', href=True):
                href = link['href']
                absolute_url = urljoin(url, href)

                if self.get_base_domain(absolute_url) == base_domain:
                    if any(pattern in absolute_url.lower() for pattern in relevant_patterns):
                        internal_links.add(absolute_url)
                        self.print_debug(f"Found relevant internal link: {absolute_url}", Fore.GREEN, "  ✓ ")

            self.print_debug(f"Found {len(internal_links)} relevant internal links", Fore.YELLOW, "📊 ")
            return list(internal_links)
        except Exception as e:
            self.print_debug(f"Error getting internal links: {str(e)}", Fore.RED, "❌ ")
            return []

    def extract_agent_info(self, url: str, html_content: str) -> Optional[RealEstateAgent]:
        self.print_debug(f"Extracting agent information from: {url}", Fore.MAGENTA, "🔍 ")
        try:
            soup = BeautifulSoup(html_content, 'html.parser')
            text_content = ' '.join(soup.stripped_strings)

            agent = RealEstateAgent()
            agent.website = url

            # Extract emails
            emails = self.patterns['email'].findall(text_content)
            if emails:
                agent.email = emails[0]
                self.print_debug(f"Found email: {agent.email}", Fore.GREEN, "  ✓ ")

            # Extract phone numbers
            phones = self.patterns['phone'].findall(text_content)
            if phones:
                phone = re.sub(r'[^\d]', '', phones[0])
                if len(phone) == 10:
                    agent.phone = f"({phone[:3]}) {phone[3:6]}-{phone[6:]}"
                    self.print_debug(f"Found phone: {agent.phone}", Fore.GREEN, "  ✓ ")

            # Extract social media
            for platform, pattern in self.patterns['social'].items():
                matches = pattern.findall(html_content)
                if matches:
                    agent.social_media[platform] = f"https://www.{matches[0]}"
                    self.print_debug(f"Found {platform} profile", Fore.GREEN, "  ✓ ")

            # Use LLM for detailed information
            self.print_debug("Using LLM to extract detailed information...", Fore.BLUE, "🤖 ")
            prompt = f"""
            Extract real estate agent information from this text:
            {text_content[:2000]}

            Return a JSON object with these fields:
            - name (full name of the agent)
            - title (professional title/role)
            - company (brokerage or company name)
            - location (city and state)
            - bio (brief professional description)
            - specialties (list of specializations)
            - languages (list of languages spoken)

            Only include information that is clearly present in the text.
            """

            try:
                llm_response = self.llm.invoke(prompt)
                agent_info = json.loads(llm_response)

                agent.name = agent_info.get('name', '')
                agent.title = agent_info.get('title', '')
                agent.company = agent_info.get('company', '')
                agent.location = agent_info.get('location', '')
                agent.bio = agent_info.get('bio', '')
                agent.specialties = agent_info.get('specialties', [])
                agent.languages = agent_info.get('languages', [])

                self.print_debug(f"Extracted agent information: {agent}", Fore.GREEN, "  ✓ ")
                return agent
            except Exception as e:
                self.print_debug(f"LLM error: {str(e)}", Fore.RED, "❌ ")
                return agent

        except Exception as e:
            self.print_debug(f"Error extracting agent info: {str(e)}", Fore.RED, "❌ ")
            return None

    def scrape(self, start_url: str) -> List[RealEstateAgent]:
        self.print_debug(f"Starting scrape at: {start_url}", Fore.CYAN, "🌐 ")
        agents = []
        to_visit = deque([(start_url, 0)])

        while to_visit:
            url, depth = to_visit.popleft()
            if depth > self.max_depth or url in self.visited_urls:
                continue

            self.visited_urls.add(url)
            try:
                response = requests.get(url, headers=self.headers)
                response.raise_for_status()
                html_content = response.text

                agents_info = self.extract_agent_info(url, html_content)
                if agents_info:
                    agents.append(agents_info)

                internal_links = self.get_internal_links(url, html_content)
                for link in internal_links:
                    if link not in self.visited_urls:
                        to_visit.append((link, depth + 1))
            except Exception as e:
                self.print_debug(f"Error visiting {url}: {str(e)}", Fore.RED, "❌ ")

        self.print_debug(f"Scraping completed. Found {len(agents)} agents.", Fore.CYAN, "🏁 ")
        return agents

def main():
    query = input("Enter search query for real estate agents: ")
    scraper = VerboseRealEstateLeadScraper(model_name="llama2")

    # Perform Google search and scrape the results
    search_results = scraper.google_search(query)

    all_agents = []
    for result_url in search_results:
        agents = scraper.scrape(result_url)
        all_agents.extend(agents)

    print("\nExtracted Agents:")
    for agent in all_agents:
        print(agent)
        print("-" * 40)

if __name__ == "__main__":
    main()


Enter search query for real estate agents: real estate people linkedin
[2024-10-05 00:33:16] 🔍 Performing Google search for: real estate people linkedin
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/services/l2/real-estate-agents
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/company/real-estate-people
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/pub/dir/Real+Estate/Agent
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/pub/dir/Real/Estate
[2024-10-05 00:33:17]   ✓ Found result: https://www.paperlesspipeline.com/blog/linkedin-real-estate-marketing
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/company/realtor
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/pub/dir/Top/Real+Estate
[2024-10-05 00:33:17]   ✓ Found result: https://www.linkedin.com/in/billgassett
[2024-10-05 00:33:17]   ✓ Found result: https://www.youtube.com/watch?v=HS3QuQHPtgU
[2024-10-05 00:33:17]   ✓ Found result: